In [3]:
#!pip install python-binance
#!pip install matplotlib --upgrade
#!pip install catboost
#!pip install --upgrade mplfinance  
#!pip install ta
#!pip install sqlalchemy
#!pip install pywhatkit

In [4]:
import numpy as np
import pandas as pd

import time
import dateparser
import pytz
import json
import math  

import datetime as dt
from datetime import datetime, timedelta

import pywhatkit as kit

from threading import Thread

from binance.exceptions import BinanceAPIException

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure

#from mpl_finance import candlestick_ochl

#from sklearn import preprocessing,model_selection,neighbors,svm
#from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
#from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

#from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from ta import add_all_ta_features

from binance.client import Client
from binance import BinanceSocketManager

In [5]:
api_id = "v7eHxeZcd4tCaGlSmrJqf7Mz11j5zJhffwZJY4QTs8fL1hugZD8hat45pZXB6Wjb"
api_key = "LsJ6TNj1s7UUfnAysLhSB034jeOh0t2qM0mZkKSbD0yLyJMlPuYaA2aqtz2hTngT"
#api_id = ""
#api_key = ""
client = Client(api_id,api_key)
    
bsm =BinanceSocketManager(client)
    
socket = bsm.trade_socket("BTCUSDT")



In [6]:
print(datetime.now())

2021-08-28 13:32:53.448334


# Indikatörler

In [7]:
def exponential_moving_average(df, n):
  EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name="EMA_" + str(n))
  df = df.join(EMA)
  return df

In [8]:
def momentum(df, n):
  M = pd.Series(df['Close'].diff(n), name="Momentum_"+str(n))
  df = df.join(M)
  return df

In [9]:
def bollinger_bands(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean())
  MSD = pd.Series(df['Close'].rolling(n, min_periods=n).std())
  b1 = 4 * MSD / MA
  B1 = pd.Series(b1, name = "BollingerB_" + str(n))
  df = df.join(B1)
  b2 = (df['Close'] - MA  + 2 * MSD) / (4 * MSD)
  B2 = pd.Series(b2, name='Bollinger%b_'+str(n))
  df = df.join(B2)
  return df

In [10]:
def stochastic_oscillator(df ,n):
  S0K = pd.Series((df['Close']- df['Low']) / (df['High']-df['Low']), name='S0k_' + str(n))
  df = df.join(S0K)
  return df

In [11]:
def relative_strength_index(df, n):
  i = 0
  UpI = [0]
  DoI = [0]

  while i + 1 <= df.index[-1]:
    UpMove = df.loc[i+1, 'High'] - df.loc[i, 'High']
    DoMove = df.loc[i+1, 'Low'] - df.loc[i, 'Low']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i + 1

  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
  df = df.join(RSI)
  return df

In [12]:
def common_channel_index(df, n):
  PP = (df['High'] + df['Low'] + df['Close']) / 3
  CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / PP.rolling(n, min_periods=n).std(), name='CCI_' + str(n))
  df = df.join(CCI)
  return df

In [13]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

# Binance Yardımcı Fonksiyonlar


In [14]:
def binanceDataFrame(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time',
                                                                 'Open',
                                                                 'High',
                                                                 'Low',
                                                                 'Close',
                                                                 'Volume',
                                                                 'Close Time',
                                                                 'Quote asset volume',
                                                                 'Number of trades',
                                                                 'Taker buy base asset volume',
                                                                 'Taker buy quote asset volume',
                                                                 'Can be ignored']);

  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

  return df

In [15]:
def date_to_miliseconds(date_str):
  epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
  d = dateparser.parse(date_str)
  if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
    d = d.replace(tzinfo=pytz.utc)

  #return difference in time
  return int((d - epoch).total_seconds()*1000.0)

In [16]:
def interval_to_miliseconds(interval):
  ms = None
  seconds_per_unit = {
      "m": 60,
      "h": 60 * 60,
      "d": 24 * 60 * 60,
      "w": 7 * 24 * 60 * 60,
  }

  unit = interval[-1]
  if unit in seconds_per_unit:
    try:
      ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
    except:
      pass

  return ms

In [17]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
  output_data=[]
  limit=500
  timeFrame = interval_to_miliseconds(interval)
  start_ts = date_to_miliseconds(start_str)
  end_ts = None
  if end_str:
    end_ts = date_to_miliseconds(end_str)
  
  idx = 0
  symbol_existed = False
  while True:
    #fetch
    temp_data = client.get_klines(symbol = symbol, 
                                  interval = interval, 
                                  limit = limit, 
                                  startTime = start_ts, 
                                  endTime = end_ts)
    
    if not symbol_existed and len(temp_data):
      symbol_existed = True

    if symbol_existed:
      output_data += temp_data

      start_ts = temp_data[len(temp_data) - 1][0] + timeFrame

    idx += 1

    if len(temp_data)<limit:
      break

    if idx % 3 == 0:
      time.sleep(1)

  return output_data

##  Sat-Satın al For Backtest

In [18]:
def satinal(bakiye,harcananpara,price):
  alinanmiktar = harcananpara/price
  kalanbakiye = bakiye - harcananpara
  return kalanbakiye,alinanmiktar

In [19]:
def sat(bakiye,satilanmiktar,price,malmiktar):
  eldeedilenpara = satilanmiktar*price
  mevcutbakiye = bakiye + eldeedilenpara
  yenimalmiktar = malmiktar - satilanmiktar
  return mevcutbakiye,yenimalmiktar

# 3 Ema Stratejisi ile Long Pozisyon 

In [20]:
## Binance'dan Verileri Alma ve Düzenleme

In [21]:

%%time
symbol = "BTCUSDT"
start = "01 Jun, 2021"
end = "01 Aug, 2021"
interval = Client.KLINE_INTERVAL_5MINUTE
klines = get_historical_klines(symbol,interval,start,end)
klines = np.array(klines)
df = binanceDataFrame(klines)


interval2 = Client.KLINE_INTERVAL_1HOUR
klines2 = get_historical_klines(symbol,interval2,start,end)
klines2 = np.array(klines2)
df2 = binanceDataFrame(klines2)


remove_cols = [c for c in df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)
df2.drop(remove_cols, axis=1, inplace=True)


df = exponential_moving_average(df,7)
df = exponential_moving_average(df,13)
df = exponential_moving_average(df,26)

df2 = exponential_moving_average(df2,7)
df2 = exponential_moving_average(df2,13)
df2 = exponential_moving_average(df2,26)


df2.drop(index=df2.index[:26], 
        axis=0, 
        inplace=True)

df.drop(index=df.index[:26], 
        axis=0, 
        inplace=True)

Wall time: 30.5 s


In [22]:
#1 saatlik grafikte grafıktekı mumların ardısık olarak ema_7'nın ustunde kapattıgı zaman dılımlerınde trade yapmak ıstıyoruz 
#o yuzden bu donemlerı tespıt edıyoruz
Trend = pd.Series(df2["Close"] - df2["EMA_7"] ,name = "Trend")
df2=df2.join(Trend)

test =df2[df2["Trend"] >10]
test = test[test["EMA_7"]>test["EMA_26"]]

timeintervallong=[]
for i in range (test.index.size - 2):
    if((test.iloc[i+1]["Open Time"] - test.iloc[i]["Open Time"]).seconds == 3600):
        timeintervallong.append((test.iloc[i]["Open Time"]))
        if((test.iloc[i+2]["Open Time"] - test.iloc[i+1]["Open Time"]).seconds != 3600):
            timeintervallong.append((test.iloc[i+1]["Open Time"]))
            
longtimeinterval=[]
longtimeinterval.append(timeintervallong[0])
for i in range (len(timeintervallong) - 2):
    if(timeintervallong[i+1].hour - timeintervallong[i].hour == 1 and timeintervallong[i+2].hour - timeintervallong[i+1].hour != 1):
        longtimeinterval.append((timeintervallong[i+1]))
        longtimeinterval.append((timeintervallong[i+2]))
longtimeinterval.append(timeintervallong[len(timeintervallong)-1])
print(len(longtimeinterval)) 

166


In [23]:
#5 DKlık grafiği 1 Saatlikte buldugumuz aralıkları kullanarak parçalıyoruz
df.set_index("Open Time",inplace=True)

d=pd.to_datetime(longtimeinterval)
e=list(range(len(longtimeinterval)-1))
df["longpossible"]=pd.cut(df.index,bins=d,labels=e,include_lowest=True)
df.dropna(inplace=True)

In [24]:
#Bu Şekilde Parcalayınca tek sayılar bizim istemedıgımız aralıklar ciftler ise bizim kullanacagımız aralıkları saglıyor.
df["longpossible"]=df["longpossible"].astype(int)
print(type(df.iloc[1]["longpossible"]))
x =df["longpossible"] % 2 == 0
df=df[x]
df = df[df["Close"] > df["EMA_26"]]
df = df[df["EMA_7"]>df["EMA_13"]]  
df = df[df["EMA_13"]>df["EMA_26"]]
df

<class 'numpy.int32'>


,Open,High,Low,Close,Volume,Close Time,EMA_7,EMA_13,EMA_26,longpossible
Open Time,,,,,,,,,,
2021-06-02 05:00:00,36662.22,36800.00,36662.22,36721.89,312.345964,2021-06-02 05:04:59.999000064,36663.174249,36630.531279,36568.583559,0
2021-06-02 05:05:00,36721.89,36789.01,36584.88,36604.82,258.298131,2021-06-02 05:09:59.999000064,36648.585687,36626.858239,36571.267739,0
2021-06-02 05:40:00,36614.79,36753.17,36578.16,36737.61,237.334120,2021-06-02 05:44:59.999000064,36627.558874,36615.585157,36583.819215,0
2021-06-02 05:45:00,36735.77,36921.00,36716.95,36890.14,490.257905,2021-06-02 05:49:59.999000064,36693.204155,36654.807277,36606.509643,0
2021-06-02 05:50:00,36889.52,36905.85,36777.75,36798.23,170.878063,2021-06-02 05:54:59.999000064,36719.460616,36675.296238,36620.711151,0
...,...,...,...,...,...,...,...,...,...,...
2021-07-31 21:40:00,41814.75,41860.95,41737.57,41850.44,113.758420,2021-07-31 21:44:59.999000064,41744.568022,41674.515132,41572.646977,164
2021-07-31 21:45:00,41850.47,41855.81,41782.24,41810.90,127.426496,2021-07-31 21:49:59.999000064,41761.151017,41693.998685,41590.295349,164
2021-07-31 21:50:00,41810.90,41984.73,41810.90,41940.31,445.137585,2021-07-31 21:54:59.999000064,41805.940763,41729.186015,41616.222361,164


## Backtest Yapma

In [25]:
#Bu stratejiyi x tarıhten y tarıhine kadar uygulasaydık ne olurduyu test edıyoruz
bakiye = 10.0
malmiktar = 0.0
maldayiz = False
karsayısı = 0
zararsayısı = 0
trendsayısı = 0;
carpan = 0.004
for i in range(1,df.index.size - 1):
  low1 = df.iloc[i]["Low"]
  high1 = df.iloc[i]["High"]
  open1 = df.iloc[i]["Open"]
  close1 = df.iloc[i]["Close"]
  ema7 = df.iloc[i]["EMA_7"]
  longpossible = df.iloc[i]["longpossible"]
  tarih = df.index[i] 
  mumunustkismi = low1 + 3*((high1 - low1) / 5)
  #çekic sekli olusmussanın if versiyonu
  if(low1 < ema7 and close1 > mumunustkismi and open1 > mumunustkismi and maldayiz==False):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = True
    zararkes = low1 - low1*carpan
    karal = close1 + 2*(close1 - zararkes)
    print("alım yapıldı kar yeri",karal,"zarar yeri",zararkes,"cekic ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
    print(close1)
   #yutan boga olusumunun if versiyonu
  elif(df.iloc[i -1]["Close"] < df.iloc[i-1]["Open"] and close1 > open1 and low1 < ema7 and maldayiz == False and close1 > df.iloc[i-1]["Open"] and ((df.iloc[i-1]["Open"] - df.iloc[i-1]["Close"]) < (close1 - open1))):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = True
    zararkes = open1 - open1*carpan
    karal = close1 + 2*(close1 - zararkes)
    print("alım yapıldı kar yeri",karal,"zarar yeri",zararkes,"yutan boga ıle yapıldı ",bakiye*malmiktar," tarih ",tarih)
    print(close1)
  if(df.iloc[i+1]["longpossible"] != longpossible and maldayiz == True):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = False
    print("ürün trend değiştiği için satıldı ",close1,"yeni bakiye",bakiye," tarih ",tarih)
    trendsayısı += 1
  if(maldayiz == True and high1 >= karal and low1 > zararkes):
    bakiye,malmiktar = sat(bakiye,malmiktar,karal,malmiktar)
    maldayiz = False
    print("ürün karla satıldı ",bakiye," tarih ",tarih)
    print(karal)
    karsayısı += 1
  elif(maldayiz == True and high1 < karal and low1 <= zararkes):
    bakiye,malmiktar = sat(bakiye,malmiktar,zararkes,malmiktar)
    maldayiz = False
    print("ürün zararla satıldı ",bakiye," tarih ",tarih)
    print(zararkes)
    zararsayısı += 1
if(bakiye == 0):
    print("bakiye",malmiktar*close1)
else:
    print("bakiye",bakiye)
print("karsayısı",karsayısı)
print("zararsayısı",zararsayısı)
print("trend ",trendsayısı)

alım yapıldı kar yeri 37276.16832 zarar yeri 36468.33084 yutan boga ıle yapıldı  0.0  tarih  2021-06-02 05:40:00
36737.61
ürün karla satıldı  10.146595905395044  tarih  2021-06-02 07:30:00
37276.16832
alım yapıldı kar yeri 37781.93904000001 zarar yeri 36939.030479999994 cekic ıle yapıldı  0.0  tarih  2021-06-02 07:55:00
37220.0
ürün karla satıldı  10.299786887725666  tarih  2021-06-02 13:30:00
37781.93904000001
alım yapıldı kar yeri 37979.46000000001 zarar yeri 37305.18 cekic ıle yapıldı  0.0  tarih  2021-06-02 14:00:00
37529.94
ürün karla satıldı  10.423153996806322  tarih  2021-06-02 14:35:00
37979.46000000001
alım yapıldı kar yeri 38410.82855999999 zarar yeri 37702.15572 yutan boga ıle yapıldı  0.0  tarih  2021-06-02 15:00:00
37938.38
ürün trend değiştiği için satıldı  38015.0 yeni bakiye 10.444204501841996  tarih  2021-06-02 19:00:00
alım yapıldı kar yeri 37960.769759999996 zarar yeri 37293.945120000004 yutan boga ıle yapıldı  0.0  tarih  2021-06-03 03:30:00
37516.22
ürün karla sat

alım yapıldı kar yeri 40660.32024 zarar yeri 39960.79488 yutan boga ıle yapıldı  0.0  tarih  2021-06-15 16:45:00
40193.97
ürün karla satıldı  13.153555564520175  tarih  2021-06-15 17:55:00
40660.32024
alım yapıldı kar yeri 40658.425680000015 zarar yeri 40103.40216 yutan boga ıle yapıldı  0.0  tarih  2021-06-16 05:05:00
40288.41
ürün trend değiştiği için satıldı  40225.01 yeni bakiye 13.1328564249217  tarih  2021-06-16 06:00:00
alım yapıldı kar yeri 40277.27735999999 zarar yeri 39085.20132 yutan boga ıle yapıldı  0.0  tarih  2021-06-17 09:50:00
39482.56
ürün trend değiştiği için satıldı  39450.0 yeni bakiye 13.122026179740146  tarih  2021-06-17 10:00:00
alım yapıldı kar yeri 36147.0 zarar yeri 35358.0 cekic ıle yapıldı  0.0  tarih  2021-06-20 21:10:00
35621.0
ürün trend değiştiği için satıldı  35650.1 yeni bakiye 13.132746006859835  tarih  2021-06-20 22:55:00
alım yapıldı kar yeri 33599.19 zarar yeri 32678.76 yutan boga ıle yapıldı  0.0  tarih  2021-06-21 14:40:00
32985.57
ürün zararla 

ürün karla satıldı  15.361848988552905  tarih  2021-07-03 07:30:00
33986.82768
alım yapıldı kar yeri 34908.76056 zarar yeri 34312.26972 yutan boga ıle yapıldı  0.0  tarih  2021-07-03 08:30:00
34511.1
ürün trend değiştiği için satıldı  34678.0 yeni bakiye 15.436140813391566  tarih  2021-07-03 15:00:00
alım yapıldı kar yeri 35170.426320000006 zarar yeri 34462.38684 yutan boga ıle yapıldı  0.0  tarih  2021-07-03 17:35:00
34698.4
ürün trend değiştiği için satıldı  34723.98 yeni bakiye 15.447520487440126  tarih  2021-07-03 20:00:00
alım yapıldı kar yeri 34910.868720000006 zarar yeri 34424.090639999995 cekic ıle yapıldı  0.0  tarih  2021-07-04 03:00:00
34586.35
ürün karla satıldı  15.592462338076517  tarih  2021-07-04 05:40:00
34910.868720000006
alım yapıldı kar yeri 35567.06736 zarar yeri 34960.516319999995 yutan boga ıle yapıldı  0.0  tarih  2021-07-04 06:10:00
35162.7
ürün karla satıldı  15.771774018679757  tarih  2021-07-04 07:50:00
35567.06736
alım yapıldı kar yeri 35937.09312 zarar yer

ürün karla satıldı  18.26575758549254  tarih  2021-07-24 06:30:00
33897.65016
alım yapıldı kar yeri 34247.35439999998 zarar yeri 33669.082800000004 cekic ıle yapıldı  0.0  tarih  2021-07-24 06:50:00
33861.84
ürün trend değiştiği için satıldı  33889.09 yeni bakiye 18.28045678359296  tarih  2021-07-24 07:00:00
alım yapıldı kar yeri 34261.366559999995 zarar yeri 33625.77672 yutan boga ıle yapıldı  0.0  tarih  2021-07-24 10:15:00
33837.64
ürün trend değiştiği için satıldı  33824.1 yeni bakiye 18.27314192993148  tarih  2021-07-24 11:15:00
alım yapıldı kar yeri 34204.70160000001 zarar yeri 33716.0442 yutan boga ıle yapıldı  0.0  tarih  2021-07-24 15:00:00
33878.93
ürün karla satıldı  18.448852044847772  tarih  2021-07-24 16:25:00
34204.70160000001
alım yapıldı kar yeri 34615.33416000001 zarar yeri 34097.33291999999 cekic ıle yapıldı  0.0  tarih  2021-07-24 17:50:00
34270.0
ürün trend değiştiği için satıldı  34343.46 yeni bakiye 18.48839837315867  tarih  2021-07-24 18:00:00
alım yapıldı kar y

# 3 Ema Stratejisi ile Short Pozisyon

%%time
symbol = "BTCUSDT"
start = "01 Jul, 2021"
end = "16 Aug, 2021"
interval = Client.KLINE_INTERVAL_5MINUTE
klines = get_historical_klines(symbol,interval,start,end)
klines = np.array(klines)
df = binanceDataFrame(klines)


interval2 = Client.KLINE_INTERVAL_1HOUR
klines2 = get_historical_klines(symbol,interval2,start,end)
klines2 = np.array(klines2)
df2 = binanceDataFrame(klines2)


remove_cols = [c for c in df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)
df2.drop(remove_cols, axis=1, inplace=True)


df = exponential_moving_average(df,7)
df = exponential_moving_average(df,13)
df = exponential_moving_average(df,26)

df2 = exponential_moving_average(df2,7)
df2 = exponential_moving_average(df2,13)
df2 = exponential_moving_average(df2,26)


df2.drop(index=df2.index[:26], 
        axis=0, 
        inplace=True)

df.drop(index=df.index[:26], 
        axis=0, 
        inplace=True)

Trend = pd.Series(df2["Close"] - df2["EMA_7"] ,name = "Trend")
df2=df2.join(Trend)

test =df2[df2["Trend"] < 10]
test =test[test["EMA_7"]<test["EMA_13"]]
test = test[test["EMA_13"]<test["EMA_26"]]


timeintervallong=[]
for i in range (test.index.size - 2):
    if((test.iloc[i+1]["Open Time"] - test.iloc[i]["Open Time"]).seconds == 3600):
        timeintervallong.append((test.iloc[i]["Open Time"]))
        if((test.iloc[i+2]["Open Time"] - test.iloc[i+1]["Open Time"]).seconds != 3600):
            timeintervallong.append((test.iloc[i+1]["Open Time"]))
            
longtimeinterval=[]
longtimeinterval.append(timeintervallong[0])
for i in range (len(timeintervallong) - 2):
    if(timeintervallong[i+1].hour - timeintervallong[i].hour == 1 and timeintervallong[i+2].hour - timeintervallong[i+1].hour != 1):
        longtimeinterval.append((timeintervallong[i+1]))
        longtimeinterval.append((timeintervallong[i+2]))
longtimeinterval.append(timeintervallong[len(timeintervallong)-1])
print(len(longtimeinterval)) 

df.set_index("Open Time",inplace=True)

d=pd.to_datetime(longtimeinterval)
e=list(range(len(longtimeinterval)-1))
df["longpossible"]=pd.cut(df.index,bins=d,labels=e,include_lowest=True)
df.dropna(inplace=True)

df["longpossible"]=df["longpossible"].astype(int)
print(type(df.iloc[1]["longpossible"]))
x =df["longpossible"] % 2 == 0
df=df[x]
df = df[df["Close"] < df["EMA_26"]]
df = df[df["EMA_7"]<df["EMA_13"]]  
df = df[df["EMA_13"]<df["EMA_26"]]
df

#Bu stratejiyi x tarıhten y tarıhine kadar uygulasaydık ne olurduyu test edıyoruz
#ÇARPAN
carpan = 0.004

bakiye = 0.0
malmiktar = 0.01
maldayiz = False
karsayısı = 0
zararsayısı = 0
trendsayısı = 0;

for i in range(1,df.index.size - 1):
  low1 = df.iloc[i]["Low"]
  high1 = df.iloc[i]["High"]
  open1 = df.iloc[i]["Open"]
  close1 = df.iloc[i]["Close"]
  ema7 = df.iloc[i]["EMA_7"]
  longpossible = df.iloc[i]["longpossible"]
  tarih = df.index[i] 
  mumunaltkismi = high1 - 3*((high1 - low1) / 5)
  #çekic sekli olusmussanın if versiyonu
  if(high1 > ema7 and close1 < mumunaltkismi and open1 < mumunaltkismi and maldayiz==False ):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = True
    zararkes = high1 + high1*carpan
    karal = close1 - 2*(zararkes - close1)
    print(close1,"den satıs yapıldı kar yeri",karal,"zarar yeri",zararkes,"cekic ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
   #yutan boga olusumunun if versiyonu
  elif(high1 > ema7 and maldayiz == False and ((df.iloc[i-1]["Open"]>open1 and df.iloc[i-1]["Close"]<close1 and df.iloc[i-1]["Close"]>open1) or (df.iloc[i-1]["Close"]>open1 and df.iloc[i-1]["Open"]<close1 and df.iloc[i-1]["Open"]>open1))):
    bakiye,malmiktar = sat(bakiye,malmiktar,close1,malmiktar)
    maldayiz = True
    zararkes = high1 + high1*carpan
    karal = close1 - 2*(zararkes - close1)
    print(close1,"den satıs yapıldı kar yeri",karal,"zarar yeri",zararkes,"yutan boga ıle yapıldı ",close1*malmiktar," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
  if(df.iloc[i+1]["longpossible"] != longpossible and maldayiz == True):
    bakiye,malmiktar = satinal(bakiye,bakiye,close1)
    maldayiz = False
    print("ürün trend değiştiği için alındı ",close1,"yeni bakiye",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    trendsayısı += 1
  if(maldayiz == True and high1 < zararkes and low1 <= karal):
    bakiye,malmiktar = satinal(bakiye,bakiye,karal)
    maldayiz = False
    print(karal,"den ürün karla alındı ",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    karsayısı += 1
  elif(maldayiz == True and high1 >= zararkes and low1 > karal):
    bakiye,malmiktar = satinal(bakiye,bakiye,zararkes)
    maldayiz = False
    print(zararkes,"den ürün zararla alındı ",bakiye," tarih ",tarih)
    
    print("bakiye ",bakiye,"malmiktar ",malmiktar)
    zararsayısı += 1

print("bakiye",bakiye)
print("malmiktarı ",malmiktar)
print("karsayısı",karsayısı)
print("zararsayısı",zararsayısı)
print("trend ",trendsayısı)
#SONUCLAR BURADA GOZULUYOR

# 3 Ema İle Güncel Olarak Al-Sat Yapma

In [ ]:

maldayiz = False
maldayiz_short = False
carpan = 0.004
carpanshort = 0.004
karzararorani = 2
karzararoranishort = 2
trendsayisi = 0
karsayisi = 0
zararsayisi = 0
bakiye = 0.0
malmiktar = 0.0
qty = 10
alimyeri = 0.0
time_change = dt.timedelta(hours = 30)

while(True):
    
    time.sleep(2)
    new_time = datetime.now()-time_change
    new_time_str = new_time.strftime("%d %b, %Y")
    
    try:
        symbol = "BTCUSDT"
        start = new_time_str
        interval = Client.KLINE_INTERVAL_5MINUTE
        klines = get_historical_klines(symbol,interval,start)
        klines = np.array(klines)
        five_min_df = binanceDataFrame(klines)
    except BinanceAPIException as e:
        print(e,"5 dklık hata vermisti ama ben 60 saniye bekleyıp tekrar denıyecegım")
        time.sleep(60)
        symbol = "BTCUSDT"
        start = new_time_str
        interval = Client.KLINE_INTERVAL_5MINUTE
        klines = get_historical_klines(symbol,interval,start)
        klines = np.array(klines)
        five_min_df = binanceDataFrame(klines)
        
    remove_cols = [c for c in five_min_df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
        
    five_min_df.drop(remove_cols, axis=1, inplace=True)
    
    five_min_df = exponential_moving_average(five_min_df,7)
    five_min_df = exponential_moving_average(five_min_df,13)
    five_min_df = exponential_moving_average(five_min_df,26)
        
    five_min_df.drop(index=five_min_df.index[:26], 
            axis=0, 
            inplace=True)
        
    try:       
        symbol = "BTCUSDT"
        start = new_time_str
        interval2 = Client.KLINE_INTERVAL_1HOUR
        klines2 = get_historical_klines(symbol,interval2,start)
        klines2 = np.array(klines2)
        one_hour_df = binanceDataFrame(klines2)
        
    except BinanceAPIException as e:
        print(e,"1 saatlik hata vermisti ama ben 60 saniye bekleyıp tekrar denıyecegım")
        time.sleep(60)
        symbol = "BTCUSDT"
        start = new_time_str
        interval2 = Client.KLINE_INTERVAL_1HOUR
        klines2 = get_historical_klines(symbol,interval2,start)
        klines2 = np.array(klines2)
        one_hour_df = binanceDataFrame(klines2)
        
    remove_cols = [c for c in one_hour_df.columns if c not in ['Open Time','Close Time','Open','Close','High','Low','Volume']]
        
    one_hour_df.drop(remove_cols, axis=1, inplace=True)

    one_hour_df = exponential_moving_average(one_hour_df,7)
    one_hour_df = exponential_moving_average(one_hour_df,13)
    one_hour_df = exponential_moving_average(one_hour_df,26)


    one_hour_df.drop(index=one_hour_df.index[:26], 
            axis=0, 
            inplace=True)

    
    
 #////////////////////////////////////////////////////////////////////////////////////////////   
    mum_kapanıs_kontrol = one_hour_df.iloc[one_hour_df.index.size - 2]["Close"] - one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_7"]
    ema_kontrol_1 = one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_7"] - one_hour_df.iloc[one_hour_df.index.size- 2]["EMA_26"]
 
    
    mum_kapanis_kontrol2 = five_min_df.iloc[five_min_df.index.size - 2]["Close"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_26"]
    ema_kontrol_5min_1 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_7"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_13"]
    ema_kontrol_5min_2 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_13"] - five_min_df.iloc[five_min_df.index.size- 2]["EMA_26"]
    
    
    trend_kontrol = False
    trend_kontrol_5min = False
    
    trend_kontrol_short = False
    trend_kontrol_short_5min = False
    
    if(mum_kapanıs_kontrol > 0 and ema_kontrol_1 > 0 ):
        trend_kontrol = True
    if(mum_kapanis_kontrol2 > 0 and ema_kontrol_5min_1 > 0 and ema_kontrol_5min_2 > 0):
        trend_kontrol_5min = True
    if(mum_kapanıs_kontrol < 0 and ema_kontrol_1 < 0 ):
        trend_kontrol_short = True
    if(mum_kapanis_kontrol2 < 0 and ema_kontrol_5min_1 < 0 and ema_kontrol_5min_2 < 0):
        trend_kontrol_short_5min = True
        
        
    son_eleman_index = five_min_df.index.size + 25
    low1 = five_min_df.iloc[five_min_df.index.size - 2]["Low"]
    high1 = five_min_df.iloc[five_min_df.index.size - 2]["High"]
    open1 = five_min_df.iloc[five_min_df.index.size - 2]["Open"]
    close1 = five_min_df.iloc[five_min_df.index.size - 2]["Close"]
    ema7 = five_min_df.iloc[five_min_df.index.size - 2]["EMA_7"]
    tarih = five_min_df.iloc[five_min_df.index.size - 2] 
    mumunustkismi = low1 + 3*((high1 - low1) / 5)
    mumunaltkismi = high1 - 3*((high1 - low1) / 5)
    
    
    
    balance = client.get_asset_balance(asset='USDT')
    orders = client.get_open_orders(symbol='BTCUSDT')
 #///////////////////////////////////////////////////////////////////////////////////////////// 
  #çekic sekli olusmussanın if versiyonu
    if(trend_kontrol == True and trend_kontrol_5min == True and low1 < ema7 and close1 > mumunustkismi and open1 > mumunustkismi and maldayiz==False and trend_kontrol == True):
        #////////////////////////////
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = True
        alimyeri = close1
        zararkes = low1 - low1*carpan
        karal = close1 + karzararorani*(close1 - zararkes)
        kit.sendwhatmsg_instantly('+905308836560', 'BUY')
        print("alim yönlü işlem yapıldı alim yeri ",close1," kar yeri",karal," zarar yeri ",zararkes," cekic ıle yapıldı ",balance," tarih ",tarih)
   #yutan boga olusumunun if versiyonu
    elif(trend_kontrol == True and trend_kontrol_5min == True and low1 < ema7 and maldayiz == False and five_min_df.iloc[five_min_df.index.size - 3]["Open"] > five_min_df.iloc[five_min_df.index.size - 3]["Close"]  and open1 < close1 and five_min_df.iloc[five_min_df.index.size - 3]["Open"]<close1 and (five_min_df.iloc[five_min_df.index.size - 3]["Open"] - five_min_df.iloc[five_min_df.index.size - 3]["Close"]) < (close1 - open1)):
        #////////////////////////////
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = True
        alimyeri = close1
        zararkes = open1 - open1*carpan
        karal = close1 + karzararorani*(close1 - zararkes)
        kit.sendwhatmsg_instantly('+905308836560', 'BUY')
        print("alim yönlü işlem yapıldı alim yeri ",close1," kar yeri ",karal," zarar yeri ",zararkes," yutan boga ıle yapıldı ",balance," tarih ",tarih)
    if(trend_kontrol == False and maldayiz == True):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        trendorani = ((alimyeri - close1) / close1 * 100) - 0.1
        kit.sendwhatmsg_instantly('+905308836560', 'CLOSE TREND')
        print("alim yönlü işlem yapıldı ürün trend değiştiği için satıldı ", close1 ,"kar veya zararı %",trendorani," yeni bakiye ", balance ," tarih ",tarih)
        trendsayisi += 1
    elif(maldayiz == True and five_min_df.iloc[five_min_df.index.size - 1]["High"] >= karal and five_min_df.iloc[five_min_df.index.size - 1]["Low"] > zararkes):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        karorani = ((karal - alimyeri) / alimyeri * 100) - 0.1
        kit.sendwhatmsg_instantly('+905308836560', 'TAKE PROFIT')
        print("alim yönlü işlem yapıldı ürün karla ",karal ," fiyattan satıldı kar oranı %",karorani,balance," tarih ",tarih)
        karsayisi += 1
    elif(maldayiz == True and five_min_df.iloc[five_min_df.index.size - 1]["High"] < karal and five_min_df.iloc[five_min_df.index.size - 1]["Low"] <= zararkes):
        #////////////////////////////
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz = False
        zararorani = ((alimyeri - zararkes) / zararkes * 100) + 0.1
        kit.sendwhatmsg_instantly('+905308836560', 'STOP LOSS')
        print("alim yönlü işlem yapıldı ürün zararla ",zararkes," fiyattan satıldı zarar oranı %",zararorani,bakiye," tarih ",tarih)
        
        
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    if(high1 > ema7 and close1 < mumunaltkismi and open1 < mumunaltkismi and maldayiz_short==False and trend_kontrol_short == True and trend_kontrol_short_5min == True ):
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz_short = True
        zararkes_short = high1 + high1*carpanshort
        karal_short = close1 - karzararoranishort*(zararkes_short - close1)
        kit.sendwhatmsg_instantly('+905308836560', 'SELL')
        print("satis yonlu işlem yapıldı satis yeri ",close1," kar yeri",karal_short," zarar yeri ",zararkes_short," cekic ıle yapıldı ",balance," tarih ",tarih)
    #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(trend_kontrol_short == True and trend_kontrol_short_5min == True and high1 > ema7 and maldayiz_short == False and five_min_df.iloc[five_min_df.index.size - 3]["Close"] > five_min_df.iloc[five_min_df.index.size - 3]["Open"]  and close1 < open1 and five_min_df.iloc[five_min_df.index.size - 3]["Open"]>close1 and (five_min_df.iloc[five_min_df.index.size - 3]["Close"] - five_min_df.iloc[five_min_df.index.size - 3]["Open"]) < (open1- close1)):
        order = client.order_market_sell(
            symbol='BTCUSDT',
            quantity=0.0004)
        #////////////////////////////
        print(order)
        maldayiz_short = True
        zararkes_short = high1 + high1*carpanshort
        karal_short = close1 - karzararoranishort*(zararkes_short - close1)
        kit.sendwhatmsg_instantly('+905308836560', 'SELL')
        print("satis yonlu işlem yapıldı satis yeri ",close1," kar yeri",karal_short," zarar yeri ",zararkes_short," yutan boga ıle yapıldı ",balance," tarih ",tarih)
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(trend_kontrol_short == False and maldayiz_short == True):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        kit.sendwhatmsg_instantly('+905308836560', 'CLOSE TREND')
        print("satis yonlu işlem yapıldı ürün trend değiştiği için alındı yeni bakiye",balance," tarih ",tarih)
    #//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(maldayiz_short == True and high1 < zararkes_short and low1 <= karal_short):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        kit.sendwhatmsg_instantly('+905308836560', 'TAKE PROFIT')
        print("satis yonlu işlem yapıldı ",karal_short,"den ürün karla alındı ",balance," tarih ",tarih)
    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    elif(maldayiz_short == True and high1 >= zararkes_short and low1 > karal_short):
        order = client.order_market_buy(
            symbol='BTCUSDT',
            quantity=0.0004)
        print(order)
        maldayiz_short = False
        kit.sendwhatmsg_instantly('+905308836560', 'STOP LOSS')
        print("satis yonlu işlem yapıldı ",zararkes_short,"den ürün zararla alındı ",balance," tarih ",tarih)
        